We are given the following joint distribution over the random variables A, B, C, and D. 

| A | B | C | D | P(A,B,C,D) |
|---|---|---|---|------------|
| T | T | T | T | 0.0448     |
| T | T | T | F | 0.0252     |
| T | T | F | T | 0.0112     |
| T | T | F | F | 0.0588     |
| T | F | T | T | 0.0144     |
| T | F | T | F | 0.0144     |
| T | F | F | T | 0.096      |
| F | F | F | F | 0.216      |
| F | T | T | T | 0.1024     |
| F | T | T | F | 0.0576     |
| F | T | F | T | 0.0256     |
| F | T | F | F | 0.1344     |
| F | F | T | T | 0.1152     |
| F | F | T | F | 0.1152     |
| F | F | F | T | 0.0768     |
| F | F | F | F | 0.1728     |

 1. Are A and B independent? 
 2. Are A and C independent?
 3. Are A and C independent given B?
 4. Are A and D independent?
 5. Are A and D independent given C?


In [17]:
import itertools
from functools import reduce
from operator import mul

import numpy as np
from pgmpy.factors.discrete import JointProbabilityDistribution

from pgmpy.factors.discrete import DiscreteFactor
from pgmpy.independencies import Independencies

# Define the variables and their states
variables = ['A', 'B', 'C', 'D']
cardinality = [2, 2, 2, 2]  # Two states for each variable: True and False

# Define the probabilities corresponding to the combinations in your table
probabilities = [
    0.0448, 0.0252, 0.0112, 0.0588,
    0.0144, 0.0144, 0.0096, 0.0216,
    0.1024, 0.0576, 0.0256, 0.1344,
    0.1152, 0.1152, 0.0768, 0.1728
]

# Create the Joint Probability Distribution
jpd_table = JointProbabilityDistribution(variables, cardinality, np.array(probabilities))

print(jpd_table)
jpd_table.marginal_distribution(['x1', 'x2'])
jpd_table.check_independence(['I'], ['D'])

+------+------+------+------+--------------+
| A    | B    | C    | D    |   P(A,B,C,D) |
+======+======+======+======+==============+
| A(0) | B(0) | C(0) | D(0) |       0.0448 |
+------+------+------+------+--------------+
| A(0) | B(0) | C(0) | D(1) |       0.0252 |
+------+------+------+------+--------------+
| A(0) | B(0) | C(1) | D(0) |       0.0112 |
+------+------+------+------+--------------+
| A(0) | B(0) | C(1) | D(1) |       0.0588 |
+------+------+------+------+--------------+
| A(0) | B(1) | C(0) | D(0) |       0.0144 |
+------+------+------+------+--------------+
| A(0) | B(1) | C(0) | D(1) |       0.0144 |
+------+------+------+------+--------------+
| A(0) | B(1) | C(1) | D(0) |       0.0096 |
+------+------+------+------+--------------+
| A(0) | B(1) | C(1) | D(1) |       0.0216 |
+------+------+------+------+--------------+
| A(1) | B(0) | C(0) | D(0) |       0.1024 |
+------+------+------+------+--------------+
| A(1) | B(0) | C(0) | D(1) |       0.0576 |
+------+--

True